# Cvičenie 5: Monte Carlo metódy

Na predošlom cvičení sme videli, ako fungujú metódy Policy Iteration a Value Iteration, ktoré ale stále predpokladali úplnú znalosť dynamiky prostredia. Na dnešnom cvičení sa pozrieme na Monte Carlo metódy, konkrétne na off-policy Monte Carlo control algoritmus odhadu $\pi^*$. Pracovať pritom budeme s rovnakým svetom gridworld.

## Metóda Monte Carlo

Na rozdiel od predošlých dvoch metód založených na dynamickom programovaní, Monte Carlo metódy sa učia z priamych interakcií s prostredím, práve preto ich priebeh závisí od konkrétnych epizód interakcie. Vďaka tomu tieto metódy vieme aplikovať aj vtedy, ak nepoznáme úplny model, teda prechody medzi stavmi nám nie sú známe, vieme ich ale nasimulovať. Algoritmus zároveň pracuje explicitne s $\varepsilon$-soft politikou, ktorej príkladom je aj $\varepsilon$-greedy politika, teda akcia s najväčšou očakávanou hodnotou sa vyberie najčastejšie, ale s istou pravdepodobnosťou bude výber akcie náhodný.

Pseudokód algoritmu nájdete na obrázku nižšie, prediskutujte jeho kroky a význam premenných a parametrov, s ktorými algoritmus pracuje.

<img src="lab05/monte_carlo.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Následne algoritmus aplikujeme pri riešení už známeho príkladu gridworld:

<img src="lab05/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede sveta. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra ukončí. Pre ostatné kroky dostane agent odmenu -1. Pre stav označený $s_{32}$ môže byť odmena 5 s pravdepodobnosťou *0.1*.

Väčšina sveta je úplne deterministická. Na pozícii $s_{12}$ fúka silný vietor, ktorý môže agenta posunúť na juh aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na juh je $0.4$. Ak agent vyberie pohyb na juh, určite sa tam dostane.

Discount factor $\gamma = 0.8$. Politika *b* pri generovaní epizód nech je náhodná politika $b(a|s) = 0.25 \ \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$  Prvý odhad hodnoty pre každú dvojicu stav-akcia bude:

```
[[0.6 0.7 0.2 0.9]
 [0.3 0.9 0.5 0.4]
 [0.6 0.7 0.1 0.0]
 [0.9 0.5 0.1 0.1]
 [0.1 0.2 0.6 0.7]
 [0.7 0.6 0.5 0.4]
 [0.9 0.2 0.3 0.7]
 [0.9 0.6 0.4 0.1]
 [0.9 0.5 0.4 0.1]]
```

Teda:

$\pi(s_{11}) = W$; $\pi(s_{12}) = E$; $\pi(s_{13}) = E$; $\pi(s_{21}) = N$; $\pi(s_{22}) = W$; $\pi(s_{23}) = N$; $\pi(s_{31}) = N$; $\pi(s_{32}) = N$; $\pi(s_{33}) = N$

## Ukážka fungovania algoritmu

Majme epizódu:

$s_{33}, E, -1, s_{33}, S, -1, s_{33}, E, -1, s_{33}, N, -1, s_{23}, E, -1, s_{23}, N, 10$,

ktorá predstavuje úspešnú interakciu, keďže agent sa dostal na cieľovú pozíciu.

Nastavíme iniciálne hodnoty premenným $G = 0$ a $W = 1$.

### Krok $s_{23}, N, 10$

Vyhodnotenie začneme od posledného kroku. Najprv sa vypočíta nová hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 0 + 10 = 10$

Ďalej aktualizujeme príslušné hodnoty *C* a *Q*:

$C(s_{23}, N) \leftarrow C(s_{23}, N) + W = 0 + 1 = 1$

$Q(s_{23}, N) \leftarrow Q(s_{23}, N) + W / C(s_{23}, N) \left [ G - Q(s_{23}, N) \right ]= 0.7 + 1 / 1 \left [ 10 - 0.7 \right ] = 10$

$Q(s_{23}) = \{ 10; 0.6; 0.5; 0.4 \}$

Akcia s najväčšou očakávanou hodnotou je teda $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{23}) = 1$

$\pi(E|s_{23}) = 0$

$\pi(S|s_{23}) = 0$

$\pi(W|s_{23}) = 0$

Ďalej potrebujeme zistiť, či môžeme pokračovať v spracovaní epizódy. Keďže $\pi(s_{23}) = N$, teda vybraná akcia, môžeme aj naďalej brať do úvahy skúsenosti získané vo vygenerovanej epizóde, aktualizujeme teda:

$W \leftarrow W \frac{1}{b(N|s_{23}} = 1 \frac{1}{0.25} = 4$

### Krok $s_{23}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 10 - 1 = 7$

Potom aktualizujeme hodnoty *C* a *Q*:

$C(s_{23}, E) \leftarrow C(s_{23}, E) + W = 0 + 4 = 4$

$Q(s_{23}, E) \leftarrow Q(s_{23}, E) + W / C(s_{23}, E) \left [ G - Q(s_{23}, E) \right ]= 0.6 + 4 / 4 \left [ 7 - 0.6 \right ] = 7$

$Q(s_{23}) = \{ 10; 7; 0.5; 0.4 \}$

Akcia s najväčšou očakávanou hodnotou je teda $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{23}) = 1$

$\pi(E|s_{23}) = 0$

$\pi(S|s_{23}) = 0$

$\pi(W|s_{23}) = 0$

Ďalej potrebujeme zistiť, či môžeme pokračovať v spracovaní epizódy. Keďže $\pi(s_{23}) = N$ a vybrala sa akcia $E$, spracovanie epizódy ukončíme.

## Úloha:

Vypočítajte aktualizované hodnoty pomocou metódy Monte Carlo na nasledujúcich epizódach (svet je rovnaký):

$s_{21}, S, -1, s_{31}, E, 5, s_{32}, S, -1, s_{32}, N, -10$

$s_{31}, E, -1, s_{32}, E, -1, s_{33}, N, -1, s_{32}, N, 10$

Vaše riešenie si skontrolujte pomocou [ukážkovej implementácie metódy](lab05/mc_offpolicy_agent.zip).

## Domáca úloha

Upravte kód ukážkového riešenia (alebo vytvorte vlastný kód) tak, aby agent využíval algoritmus On-policy first-visit MC control:

<img src="lab05/mc_onpolicy.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>